In [2]:
#| default_exp in_memory_tiling

In [5]:
#| export 
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import torch
from pathlib import Path
import numpy as np
import pandas as pd
from functools import lru_cache 

Notebook mostly copied from [here](https://www.kaggle.com/code/squidinator/sennet-hoa-in-memory-tiled-dataset-pytorch)

In [3]:
def rle_decode(
        mask_rle: str,
        img_shape:tuple=None,
        )->np.ndarray:
    'decode mask from run length encoding'

    seq = mask_rle.split()
    starts = np.array(list(map(int, seq[0::2])))
    lengths = np.array(list(map(int, seq[1::2])))
    assert len(starts) == len(lengths)
    ends = starts + lengths
    img = np.zeros((np.product(img_shape),), dtype=np.uint8)
    for begin, end in zip(starts, ends):
        img[begin:end] = 1
    # https://stackoverflow.com/a/46574906/4521646
    img.shape = img_shape
    return img

In [6]:
@lru_cache(maxsize=64)
def ropen(path:Path):
    return rasterio.open(path)